### Import and Load Packages 

In [1]:
# Load packages
import requests
import pandas as pd
import numpy as np
from datetime import date
import csv
import re
import datetime
import os
from sklearn.feature_selection import VarianceThreshold
from pandas import read_csv
from sklearn.datasets import load_breast_cancer
import kaplanmeier as km
from sklearn.decomposition import PCA
import pylab as pl
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
%matplotlib inline
from qqman import qqman
import pandas as pd
from pandas_plink import read_plink
from scipy.stats import ttest_ind
import math
import os
from lifelines import KaplanMeierFitter
from pandas import DataFrame
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn import preprocessing
from numpy import set_printoptions
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import datasets
import seaborn as sns; sns.set()
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
#from xgboost import XGBClassifier
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from numpy import arange
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import pandas as pd
from google.cloud import bigquery
import os
import matplotlib.pyplot as plt
import textwrap
import re
from tqdm import tqdm
%matplotlib inline

### Set up connection to BigQuery 

In [4]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-rit-phi-starr-prod'
cdm_dataset_id = 'starr_omop_cdm5_deid_2021_12_13'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = 'starr_omop_cdm5_deid_20211213'

In [3]:
# Use correct path whether you are local or Nero
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/kbechler/adc.json' 

# Set correct Nero project
os.environ['GCLOUD_PROJECT'] = nero_gcp_project
client = bigquery.Client(project = work_project_id)

In [5]:
# Tables to copy
tables_to_copy = ["care_site",
                    "cdm_source",
                    "concept", # Big Vocab table
                    "concept_ancestor", # Big Vocabulary Table
                    "concept_class", # Big Vocabulary Table
                    "concept_relationship", # Big Vocabulary Table
                    "concept_synonym", # Vocab Table
                    "condition_era",
                    "condition_occurrence",
                    "cost", # empty 
                    "death",
                    "device_exposure",
                    "domain", # Vocab Table
                    "dose_era",
                    "drug_era",
                    "drug_exposure",
                    "drug_strength", # Vocab Table
                    "fact_relationship",                  
                    "location",
                    "measurement",
                    "metadata",
                    #"note"
                    "note_nlp", # Big Table in general
                    "observation",
                    "observation_period",
                    "payer_plan_period", # empty
                    "person",
                    "procedure_occurrence",
                    "provider",
                    "relationship", # Vocab Table
                    "source_to_concept_map", # Vocab Table
                    "specimen", # empty
                    "visit_detail",
                    "visit_occurrence",
                    "vocabulary"] # Vocab Table]

In [6]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-rit-phi-starr-prod'
cdm_dataset_id = 'starr_omop_cdm5_deid_2021_12_13'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = 'starr_omop_cdm5_deid_20211213'

In [9]:
client.create_dataset(f"{work_project_id}.{work_dataset_id}", exists_ok=True)

Dataset(DatasetReference('som-nero-nigam-starr', 'starr_omop_cdm5_deid_20211213'))

In [10]:
#%%time
for table_id in tables_to_copy:
    sql = """
        CREATE TABLE
         `{work_project_id}.{work_dataset_id}.{table_id}` AS
        SELECT * 
        FROM 
         `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
        """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'work_project_id': work_project_id,
                        'work_dataset_id': work_dataset_id,
                        'table_id': table_id})
    
    job_config = bigquery.QueryJobConfig(dry_run=False, use_query_cache=True)
    query_job = client.query(sql, job_config)
    query_job.result()
    print("This query will process {} bytes.".format(query_job.total_bytes_processed))

This query will process 296909 bytes.
This query will process 477 bytes.
This query will process 1189646926 bytes.
This query will process 2578649600 bytes.
This query will process 18284 bytes.
This query will process 3191034807 bytes.
This query will process 778895330 bytes.
This query will process 2589347328 bytes.
This query will process 21286916899 bytes.
This query will process 0 bytes.
This query will process 1726684 bytes.
This query will process 252094990 bytes.
This query will process 1978 bytes.
This query will process 0 bytes.
This query will process 2973445098 bytes.
This query will process 56169673489 bytes.
This query will process 157584097 bytes.
This query will process 1596418120 bytes.
This query will process 122295637 bytes.
This query will process 396543341272 bytes.
This query will process 40 bytes.
This query will process 2545005207620 bytes.
This query will process 1738832670564 bytes.
This query will process 473688325 bytes.
This query will process 0 bytes.
This 